<a href="https://colab.research.google.com/github/RA1911033010093/AI_Project/blob/main/Earthquake_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import datetime 
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

sns.set_style('darkgrid')

In [27]:
## reading the file from directort

df = pd.read_csv('database.csv')

In [3]:
## display top 5 rows

df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [28]:
df.shape

(23412, 21)

In [29]:
## main features that are required to predict earthquake

df = df[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]

In [30]:
df.head()

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,01/02/1965,13:44:18,19.246,145.616,131.6,6.0
1,01/04/1965,11:29:49,1.863,127.352,80.0,5.8
2,01/05/1965,18:05:58,-20.579,-173.972,20.0,6.2
3,01/08/1965,18:49:43,-59.076,-23.557,15.0,5.8
4,01/09/1965,13:32:50,11.938,126.427,15.0,5.8


In [31]:
## checking if there is any null values or not 

df.isnull().sum()

Date         0
Time         0
Latitude     0
Longitude    0
Depth        0
Magnitude    0
dtype: int64

In [32]:
# converting Date column from string to datetime

df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%m/%d/%Y')

In [33]:
## Extracting minutes, seconds, hours from column Time

minutes = []
hours = []
seconds = []

for t in df.Time:
    t  = t.split(":")
    minutes.append(t[1])
    hours.append(t[0])
    seconds.append(t[2])

In [34]:
## creating new columns minutes, hours, seconds

df['minutes'] = minutes
df['hours'] = hours
df['seconds'] = seconds

df['minutes'] = pd.to_numeric(df['minutes'], errors='coerce')
df['hours'] = pd.to_numeric(df['hours'], errors='coerce')
df['seconds'] = pd.to_numeric(df['seconds'], errors='coerce') 

In [35]:
## creating new columns year, month, day

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [36]:
## dropping date and time column from dataset

df.drop(['Date', 'Time'], axis=1, inplace=True)

In [37]:
df.head()

,Latitude,Longitude,Depth,Magnitude,minutes,hours,seconds,year,month,day
0,19.246,145.616,131.6,6.0,44,13.0,18.0,1965.0,1.0,2.0
1,1.863,127.352,80.0,5.8,29,11.0,49.0,1965.0,1.0,4.0
2,-20.579,-173.972,20.0,6.2,5,18.0,58.0,1965.0,1.0,5.0
3,-59.076,-23.557,15.0,5.8,49,18.0,43.0,1965.0,1.0,8.0
4,11.938,126.427,15.0,5.8,32,13.0,50.0,1965.0,1.0,9.0


In [38]:
df.dropna(inplace=True)

In [39]:
df.isnull().sum()

Latitude     0
Longitude    0
Depth        0
Magnitude    0
minutes      0
hours        0
seconds      0
year         0
month        0
day          0
dtype: int64

In [40]:
## independent and dependent features

X = df[['Latitude', 'Longitude', 'minutes', 'hours', 'seconds', 'year', 'month', 'day']]
y = df[['Depth', 'Magnitude']]

In [41]:
X = X.values
y = y.values

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16386, 8) (7023, 8) (16386, 2) (7023, 2)


In [44]:
## creating a neural Network model 

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(8,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [45]:
model.compile(optimizer='adam', loss='squared_hinge', metrics=['accuracy'])

In [46]:
## training the model

history = model.fit(X_train, y_train, batch_size=64, epochs=20, verbose=1, validation_data=(X_test, y_test))

Epoch 1/20
257/257 [==============================] - 2s 4ms/step - loss: 0.5040 - accuracy: 0.9813 - val_loss: 0.5033 - val_accuracy: 0.9825
Epoch 2/20
257/257 [==============================] - 1s 3ms/step - loss: 0.5040 - accuracy: 0.9813 - val_loss: 0.5033 - val_accuracy: 0.9825
Epoch 3/20
257/257 [==============================] - 1s 3ms/step - loss: 0.5040 - accuracy: 0.9813 - val_loss: 0.5033 - val_accuracy: 0.9825
Epoch 4/20
257/257 [==============================] - 1s 3ms/step - loss: 0.5040 - accuracy: 0.9813 - val_loss: 0.5033 - val_accuracy: 0.9825
Epoch 5/20
257/257 [==============================] - 1s 3ms/step - loss: 0.5040 - accuracy: 0.9813 - val_loss: 0.5033 - val_accuracy: 0.9825
Epoch 6/20
257/257 [==============================] - 1s 3ms/step - loss: 0.5040 - accuracy: 0.9813 - val_loss: 0.5033 - val_accuracy: 0.9825
Epoch 7/20
257/257 [==============================] - 1s 3ms/step - loss: 0.5040 - accuracy: 0.9813 - val_loss: 0.5033 - val_accuracy: 0.9825
Epoch 